Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



Pela leitura da problemática do problema, e considerando que um item da mochila vai ter um peso e um valor em dinheiros, que no total será o fitness daquele indivíduo, fica claro que podemos utilizar um dicionário para criar os objetos possíveis de se colocar na mochila.

A partir deste dicionário, podemos tomar uma abordagem de que, se um objeto estiver presente na mochila, o indivíduo terá um valor de gene representando isso, e se não, terá outro valor. Isso se assemelha ao experimento das caixas binárias, logo, podemos utilizar algumas funções do experimento das caixas binárias para a construção deste algorítmo, e formar uma lista representando a mochila, que terá os valores de gene onde 1 representa que um objeto está presente naquele índice, e 0 indica que não há objetos no índice.

Podemos criar uma função que compute o valor total e o peso total de uma mochila, baseado na presença ou não dos objetos, e com isso, retornar um indivíduo contendo uma combinação de genes que represente uma combinação de objetos da mochila, com seu respectivo valor e peso, e durante o processo de seleção e cruzamento selecionar o candidato que tenha o melhor fitness baseado no melhor peso e valor da mochila.

In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
     "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [5]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf") # Colocamos -float inf para que o valor de melhor fitness ja visto seja fácilmente substituido pelo primeiro valor que for passado como fit
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dabloons "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 15500 | Peso: 13.5

Você deve pegar os seguintes itens:
+ Quadrinho super raro do Aranha-Homem da vida real
+ docinhos para o stress
+ teclado musical
+ vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 15500 dabloons e peso de 13.5 unidades de massa.


**Desafio**: resolva o experimento considerando uma busca em grade para encontrar a melhor resposta.

## Conclusão

Neste experimento nós tratamos o problema da mochila, onde queremos armazenar itens em uma mochila e ter um maior lucro no final baseado nos valores do item. Porém, não podemos considerar apenas os valores dos itens para armazenar, pois a mochila tem um ***limite*** de peso que pode suportar, logo é necessário criar uma nova função de suporte que compute o peso e o valor de cada item para o cálculo da função objetivo, verificando se o indivíduo contendo os genes que representam os itens é um indivíduo ***válido*** para ser um candidato do nosso problema.

Logo de início, adotamos a estratégia de criarmos um dicionário contendo valor e peso de cada item que pode ser colocado na mochila, e definimos que um indivíduo será composto por genes binários (ou seja, que assumem o valor 0 ou 1). Os genes representarão se um item está presente naquele individuo (1) ou não (0), e assim teremos diversos indivíduos contendo itens variados.
Foram utilizadas diversas funções do problema das caixas binárias como mutação e seleção, e esta ultima realizará a seleção baseando-se na condição se o indivíduo é válido (não excede o peso).
Na função objetivo, aplicamos uma ***restrição*** aos valores de mochila encontrados, pois se temos um valor de mochila maior que o limite de peso desta, o valor de fitness retornado será de 0.01, convidando o algoritmo a não utilizar este valor durante a seleção, pois o candidato viola a restrição.

Por fim, como temos um problema de maximização, a variável ***menor_fitness_já_visto*** tem valor -float ("inf"), para que esta seja facilmente substituida pelo primeiro valor de fitness que o algoritmo achar.

Vemos então que este algoritmo consegue resolver o problema da mochila respeitando as restrições impostas, mas pode não ser o método mais efetivo visto que realiza uma busca aleatória ao percorrer todos os candidatos e todos os genes contendo valores e pesos

## Playground



Os indivíduos são caixas binárias, com os genes assumindo valores de 0 e 1, onde 0 representa que o item não está na mochila, e 1 que o item está na mochila.
O 0.01 indica que, como temos um problema de maximização, a função retornar um fitness de 0.01 meio que convida o algoritmo a nao usar esse valor
Só queremos que o hall da fama tenha candidados que podemos usar